# How to deal with timestamps

## Install

In [1]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# !pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="pandas-test-domain-1", reset=True)

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/281e55cb3425360e464682cfb4672fcb.sqlite



# Data owner: upload dataset

In [5]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

## Load data

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

In [6]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [7]:
popcon = pd.read_csv(autocache(f"{PANDAS_DATA}/popularity-contest"), sep=' ', )[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

In [8]:
set(popcon["tag"])

{'<OLD>', '<RECENT-CTIME>', nan}

## Create mock data

Lets create a mock dataset

In [9]:
# weather_types = set(weather_2012_final.Weather.to_list())

In [10]:
from random import randint
import random

In [11]:
def get_random_timestamp():
    return "135" + "".join([str(randint(0,9)) for i in range(6)])

In [12]:
def get_mock_date(i):
    return str(parse('Jun 1 2010') + timedelta(days=i))

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [13]:
mock_functions = {'atime': lambda: get_random_timestamp(),
                  'ctime': lambda: get_random_timestamp(),
                  'package-name': lambda: random.choice([
                      'libghc-stm-dev',
                      'libqtdee-dev',
                      'geoclue-ubuntu-geoip',
                      'libdesktop-agnostic0',
                      'ubuntu-extras-keyring',
                      'libbsd0',
                      'libxres-dev']),
                  'mru-program': lambda: random.choice([
                      '/usr/bin/opam',
                      '/usr/bin/onboard',
                      '/lib/init/upstart-job',
                      '/usr/bin/page',
                  ]),
                  'tag': lambda: random.choice(['<OLD>', '<RECENT-CTIME>', 'nan'])}


In [14]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(popcon))],
                    columns=popcon.columns)
    


Upload the data

In [15]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"weather", data=popcon,
                                                       mock=mock, mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: weather


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.16it/s]


<class 'syft.service.response.SyftSuccess'>: Dataset Added

In [16]:
popcon.head()

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


## Create user account

In [17]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

# Data scientist: create result pointer

In [18]:
import numpy as np
import pandas as pd

## Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

## Get mocks

In [19]:
ds = guest_domain_client.datasets[0]

In [20]:
asset = ds.assets[0]

In [21]:
popcon = asset.mock

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [22]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,135901774,135598185,libghc-stm-dev,/usr/bin/opam,nan
1,135018202,135328344,libqtdee-dev,/usr/bin/opam,<RECENT-CTIME>
2,135338223,135642081,libqtdee-dev,/usr/bin/opam,nan
3,135811863,135882106,libdesktop-agnostic0,/lib/init/upstart-job,<RECENT-CTIME>
4,135465700,135768091,libbsd0,/lib/init/upstart-job,<RECENT-CTIME>


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [23]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [24]:
popcon

,atime,ctime,package-name,mru-program,tag
0,135901774,135598185,libghc-stm-dev,/usr/bin/opam,nan
1,135018202,135328344,libqtdee-dev,/usr/bin/opam,<RECENT-CTIME>
2,135338223,135642081,libqtdee-dev,/usr/bin/opam,nan
3,135811863,135882106,libdesktop-agnostic0,/lib/init/upstart-job,<RECENT-CTIME>
4,135465700,135768091,libbsd0,/lib/init/upstart-job,<RECENT-CTIME>
...,...,...,...,...,...
2892,135633923,135712755,libdesktop-agnostic0,/usr/bin/page,nan
2893,135049758,135467553,ubuntu-extras-keyring,/usr/bin/onboard,<OLD>
2894,135031136,135604571,libbsd0,/usr/bin/opam,nan
2895,135196124,135275111,ubuntu-extras-keyring,/usr/bin/page,<RECENT-CTIME>


In [25]:
popcon['atime'] = guest_client.api.lib.pandas.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = guest_client.api.lib.pandas.to_datetime(popcon['ctime'], unit='s')

In [27]:
# root_domain_client = node.login(email="info@openmined.org", password="changethis")
# popcon.get_from(root_domain_client)

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [30]:
# TODO: Fix
# popcon['atime'].dtype

So now we can look at our `atime` and `ctime` as dates!

In [31]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1974-04-22 22:29:34,1974-04-19 10:09:45,libghc-stm-dev,/usr/bin/opam,nan
1,1974-04-12 17:03:22,1974-04-16 07:12:24,libqtdee-dev,/usr/bin/opam,<RECENT-CTIME>
2,1974-04-16 09:57:03,1974-04-19 22:21:21,libqtdee-dev,/usr/bin/opam,nan
3,1974-04-21 21:31:03,1974-04-22 17:01:46,libdesktop-agnostic0,/lib/init/upstart-job,<RECENT-CTIME>
4,1974-04-17 21:21:40,1974-04-21 09:21:31,libbsd0,/lib/init/upstart-job,<RECENT-CTIME>


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [32]:
atime = popcon['atime']

In [40]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [41]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [42]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
603,1974-04-15 17:35:13,1974-04-24 01:22:57,ubuntu-extras-keyring,/usr/bin/opam,<OLD>
505,1974-04-14 10:04:53,1974-04-24 01:15:07,ubuntu-extras-keyring,/lib/init/upstart-job,nan
2066,1974-04-13 08:42:28,1974-04-24 00:44:59,ubuntu-extras-keyring,/usr/bin/onboard,<RECENT-CTIME>
1406,1974-04-19 04:05:43,1974-04-24 00:41:41,geoclue-ubuntu-geoip,/usr/bin/opam,<RECENT-CTIME>
894,1974-04-19 08:22:04,1974-04-24 00:16:59,geoclue-ubuntu-geoip,/usr/bin/opam,<OLD>
2415,1974-04-12 14:45:24,1974-04-24 00:11:16,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan
2578,1974-04-20 17:05:08,1974-04-23 23:31:05,ubuntu-extras-keyring,/usr/bin/page,nan
878,1974-04-18 11:09:59,1974-04-23 23:30:23,ubuntu-extras-keyring,/usr/bin/page,<RECENT-CTIME>
1821,1974-04-23 22:11:54,1974-04-23 23:17:37,ubuntu-extras-keyring,/lib/init/upstart-job,nan
1133,1974-04-18 18:32:07,1974-04-23 23:03:06,geoclue-ubuntu-geoip,/usr/bin/onboard,nan


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Requesting result

In [43]:
nonlibraries.request(guest_client)

```python
class Request:
  id: str = ca101d3997dc47d98080beb2893acb00
  requesting_user_verify_key: str = b889ddf6f8210fa4a0e2d2e5ab3a98dde658cc0102a8737a178a8874fda4cbbb
  approving_user_verify_key: str = None
  request_time: str = 2023-05-24 13:08:42
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 281e55cb3425360e464682cfb4672fcb
  request_hash: str = "526463393f48e13eea4858df7ab7816be2561bfcc2c555877b3ea54ca337b1a3"
  changes: str = [syft.service.request.request.ActionStoreChange]

```

# Data owner: accept request

In [44]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

In [45]:
root_domain_client.api.services.request[0].approve()

<class 'syft.service.response.SyftSuccess'>: Request ca101d3997dc47d98080beb2893acb00 changes applied

# Data Owner: fetch result

In [46]:
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [47]:
real_result = nonlibraries.get_from(guest_client)

In [48]:
real_result

,atime,ctime,package-name,mru-program,tag
0,2013-12-17 15:56:37,2013-05-04 02:07:40,perl-base,/usr/bin/perl,None
1,2013-12-17 15:56:36,2012-12-01 14:01:20,login,/bin/su,None
17,2013-12-17 15:55:33,2013-11-25 16:25:38,fingerprint-gui,/lib/security/pam_fingerprint-gui.so,None
18,2013-12-17 15:55:32,2012-12-01 05:53:57,dash,/bin/dash,None
19,2013-12-17 15:55:29,2012-12-01 05:54:37,popularity-contest,/usr/sbin/popularity-contest,None
...,...,...,...,...,...
2089,2011-04-30 20:36:36,2012-12-01 05:54:17,x11-xfs-utils,/usr/bin/xfsinfo,<OLD>
2090,2011-04-30 15:07:31,2012-12-01 05:54:15,dvd+rw-tools,/usr/bin/rpl8,<OLD>
2092,2010-12-12 02:51:56,2012-12-01 05:54:37,vbetool,/usr/sbin/vbetool,<OLD>
2093,2010-10-15 16:41:50,2012-12-01 05:54:37,pptp-linux,/usr/sbin/pptp,<OLD>


In [49]:
node.land()